In [11]:
import pandas as pd
import re
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig
import os
import random
import numpy as np
import sys
sys.path.append("../../TextModel/")
from utils import *
from torch.utils.data import Dataset, DataLoader
from sklearn.cluster import DBSCAN
from sklearn.decomposition import KernelPCA
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import silhouette_score
import warnings
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from surprise import SVD, NMF, Dataset, Reader
warnings.filterwarnings("ignore")

cluster.csv, director.csv, similarity.csv 전처리

In [12]:
'''
movie_id를 0~len(movie)로 설정
tmdb로 만들어진 데이터들을 MovieLens 데이터랑 title 기준으로 merge
-> 이때, inner join을 하기 때문에, title이 없는 데이터는 제거됨.
'''
print()

In [13]:
ml_movie = pd.read_csv("../../Data/MovieLens/movie.csv")

In [14]:
change_ml_title = [
"Alien 3",
"blood & wine",
"american dream",
"dumb & dumber",
"kama sutra: a tale of love",
"kicking and screaming",
"oscar & lucinda",
"richie rich",
"seven",
"willy wonka and the chocolate factory",
"up close and personal"
]

change_to = [
"Alien³",
"blood and wine",
"american dreamz",
"dumb and dumbe",
"kama sutra - a tale of love",
"kicking & screaming",
"oscar and lucinda",
"ri¢hie ri¢h",
"se7en",
"up close & personal",
"willy wonka & the chocolate factory"
]
change_title_dic = {k:v for k, v in zip(change_ml_title, change_to)}

In [15]:
ml_movie.movie_title = ml_movie.movie_title.apply(lambda x: x.lower())

In [16]:
def remove_parenthesis(text):
    cleaned_text = re.sub(r'\([^()]*\)', '', text)
    return cleaned_text.strip()

ml_movie.movie_title = ml_movie.movie_title.map(lambda x: remove_parenthesis(x))
ml_movie.rename(columns={'movie_title':'title'}, inplace=True)

def c(x):
    try:
        return change_title_dic[x]
    except:
        return x

# title 추가 변경 전처리
ml_movie.title = ml_movie.title.apply(lambda x:c(x))

In [17]:
ml_movie.head(1)

,movie_id,title,release_date,video_release_date,IMDb_URL
0,1,toy story,01-Jan-95,01-Jan-95,http://us.imdb.com/M/title-exact?Toy%20Story%2...


In [18]:
# TODO: cluster.csv, director.csv, similarity.csv 3개 파일을 title 기준으로 merge
c = pd.read_csv("./preprocess_phase_0/cluster.csv")
d = pd.read_csv("./preprocess_phase_0/director.csv")
s = pd.read_csv("./preprocess_phase_0/similarity.csv")

In [19]:
ml_movie.head(1)

,movie_id,title,release_date,video_release_date,IMDb_URL
0,1,toy story,01-Jan-95,01-Jan-95,http://us.imdb.com/M/title-exact?Toy%20Story%2...


In [20]:
merge_tmp = ml_movie[['movie_id', 'title']]

In [21]:
c.head(1)
print(len(c))
print(c.title.nunique())

4798
4798


In [22]:
d.head(1)
print(len(d))
print(d.title.nunique())
print(d.director_name.nunique())

4767
4767
2344


In [23]:
s.head(1)
print(len(s))
print(s.title.nunique())
print(s.similar_title.nunique())

1561
1561
1561


In [24]:
c = pd.merge(left=c, right=merge_tmp, on='title', how='inner')

In [25]:
c.head(1)
print(len(c))
print(c.title.nunique())

402
394


In [26]:
d = pd.merge(left=d, right=merge_tmp, on='title', how='inner')

In [27]:
d.head(1)
print(len(d))
print(d.title.nunique())
print(d.director_name.nunique())

401
393
283


In [28]:
s = pd.merge(left=s, right=merge_tmp, on='title', how='inner')

In [29]:
s.head(1)
print(len(s))
print(s.title.nunique())
print(s.similar_title.nunique())

129
128
128


In [30]:
s = s[["similar_title", "cosine_similarity", "movie_id"]]

In [31]:
s.rename(columns={"similar_title":"title"}, inplace=True)

In [32]:
s.head(1)

,title,cosine_similarity,movie_id
0,the terminator,0.901428,96


In [33]:
s = pd.merge(left=s, right=merge_tmp, on='title', how='inner')

In [34]:
s.rename(columns={"movie_id_x":"movie_id"}, inplace=True)
s.rename(columns={"movie_id_y":"similar_movie_id"}, inplace=True)

In [35]:
s.head(1)

,title,cosine_similarity,movie_id,similar_movie_id
0,mars attacks!,0.852736,257,235


In [36]:
s.head(1)
print(len(s))
print(s.title.nunique())

21
21


MovieLens와 TMDB는 각각 데이터 생성 연도가 달라서 최신화가 서로 안 이루어짐

In [37]:
c.head(1)

,title,overview,cluster_id,movie_id
0,titanic,"84 years later, a 101-year-old woman named Ros...",13,313


In [38]:
d.head(1)

,title,director_name,movie_id
0,titanic,James Cameron,313


In [39]:
s.head(1)

,title,cosine_similarity,movie_id,similar_movie_id
0,mars attacks!,0.852736,257,235


In [40]:
c.to_csv("./preprocess_phase_1/cluster_0.csv", index=False)
d.to_csv("./preprocess_phase_1/director_0.csv", index=False)
s.to_csv("./preprocess_phase_1/similarity_0.csv", index=False)
ml_movie.to_csv("./preprocess_phase_1/movie_0.csv", index=False)

user_sim.csv, item_sim.csv, movie_genres.csv, ratings.csv는 나중에 다른 csv파일이랑 같이 전처리